# Fixing pitcher issues

This notebook is an initial investigation into the issues I was having with pitchers missing/having different data in different places.

In [1]:
import pandas as pd

from utils.data_loader import Dataset

## mlb_games_df versus Dataset.load_games()

In [197]:
train_ds = Dataset(name='train')
orig_df = train_ds.load_games(start_date='2000-01-01', end_date='2005-01-01').copy()

In [2]:
mlb_games_df = pd.read_csv('../data/mlb_games_df.csv')
mlb_games_df['date'] = pd.to_datetime(mlb_games_df['date'])
mlb_games_df = mlb_games_df[mlb_games_df['date'] < '2005-01-01']

In [174]:
team_stats_df = pd.read_csv('../data/team_stats.csv')

In [175]:
team_stats_df.head()

,Year,Team,Wins,Losses,W-L-pct,Avg_Attendance,Ghome,ERA,W,E,...,SOA,WCWin,HBP,BBA,L,FP,HR,PPF,CS,HRA
0,2000.0,LAN,86.0,76.0,0.530864,35699.943750,81.0,4.10,86,135,...,1154,N,51.0,600,76,0.978,211,94,42.0,176
1,2001.0,LAN,86.0,76.0,0.530864,35078.895062,81.0,4.25,86,116,...,1212,N,56.0,524,76,0.981,206,91,42.0,184
2,2002.0,LAN,92.0,70.0,0.567901,34668.956522,81.0,3.69,92,90,...,1132,N,53.0,555,70,0.985,155,92,37.0,165
3,2003.0,LAN,85.0,77.0,0.524691,33715.166667,81.0,3.16,85,119,...,1289,N,72.0,526,77,0.981,124,94,36.0,127
4,2004.0,LAN,93.0,69.0,0.574074,37452.246914,81.0,4.01,93,73,...,1066,N,62.0,521,69,0.988,203,95,41.0,178


In [176]:
mlb_games_df.head()

,date,Y,M,D,home_team,away_team,home_win,home_pitcher,away_pitcher,home_elo,...,avg_diff,obp_diff,slg_diff,avg_pct_diff,obp_pct_diff,slg_pct_diff,home_rest,away_rest,away_team_season_game_num,home_team_season_game_num
0,2001-04-01,2001,4.0,1.0,TOR,TEX,1.0,loaizes01,helliri01,1499.563,...,-0.008060,-0.010103,0.023271,-2.947374,-2.977845,4.989568,5.0,5.0,0,0
1,2001-04-02,2001,4.0,2.0,SEA,OAK,1.0,garcifr03,hudsoti01,1519.464,...,-0.000864,0.001190,-0.016229,-0.323318,0.331871,-3.705210,5.0,5.0,0,0
2,2001-04-02,2001,4.0,2.0,NYA,KCA,1.0,clemero02,suppaje01,1529.511,...,-0.010188,0.006929,0.024787,-3.703559,1.970596,5.554343,5.0,5.0,0,0
3,2001-04-02,2001,4.0,2.0,CIN,ATL,0.0,harnipe01,burkejo03,1527.274,...,0.003972,-0.001729,0.020216,1.459194,-0.506960,4.555242,5.0,5.0,0,0
4,2001-04-02,2001,4.0,2.0,CHN,WAS,0.0,liebejo01,vazquja01,1462.510,...,-0.010158,0.009335,-0.018992,-3.996340,2.803560,-4.646432,5.0,5.0,0,0


In [177]:
assert mlb_games_df.shape[0] == train_ds.data.shape[0]

## Adding team stats

In [178]:
train_ds.add_team_stats(cols=['W-L-pct'])

,date,Y,M,D,home_team,away_team,home_win,home_pitcher,away_pitcher,home_elo,...,slg_diff,avg_pct_diff,obp_pct_diff,slg_pct_diff,home_rest,away_rest,away_team_season_game_num,home_team_season_game_num,home_W-L-pct_offset1year,away_W-L-pct_offset1year
0,2001-04-01,2001,4.0,1.0,TOR,TEX,1.0,loaizes01,helliri01,1499.562988,...,0.023271,-2.947374,-2.977845,4.989568,5.0,5.0,0,0,0.512346,0.438272
1,2001-04-02,2001,4.0,2.0,SEA,OAK,1.0,garcifr03,hudsoti01,1519.463989,...,-0.016229,-0.323318,0.331871,-3.705210,5.0,5.0,0,0,0.561728,0.565217
2,2001-04-02,2001,4.0,2.0,NYA,KCA,1.0,clemero02,suppaje01,1529.510986,...,0.024787,-3.703559,1.970596,5.554343,5.0,5.0,0,0,0.540373,0.475309
3,2001-04-02,2001,4.0,2.0,CIN,ATL,0.0,harnipe01,burkejo03,1527.274048,...,0.020216,1.459194,-0.506960,4.555242,5.0,5.0,0,0,0.524691,0.586420
4,2001-04-02,2001,4.0,2.0,CHN,WAS,0.0,liebejo01,vazquja01,1462.510010,...,-0.018992,-3.996340,2.803560,-4.646432,5.0,5.0,0,0,0.401235,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9708,2004-10-03,2004,10.0,3.0,CIN,PIT,0.0,clausbr01,perezol01,1455.857056,...,0.000173,-28.571428,-16.740494,0.045767,1.0,1.0,80,80,0.425926,0.462963
9709,2004-10-03,2004,10.0,3.0,OAK,ANA,1.0,hudsoti01,seleaa01,1528.114990,...,-0.134015,-20.261438,-17.270956,-42.810459,1.0,1.0,80,80,0.592593,0.475309
9710,2004-10-03,2004,10.0,3.0,CHN,ATL,1.0,maddugr01,byrdpa01,1536.317017,...,0.006766,0.017153,11.871174,1.509434,1.0,1.0,80,81,0.543210,0.623457
9711,2004-10-03,2004,10.0,3.0,MIN,CLE,0.0,lohseky01,leecl02,1532.850952,...,-0.031043,-5.032503,1.314286,-8.076923,1.0,1.0,80,80,0.555556,0.419753


In [179]:
assert train_ds.data.shape[0] == mlb_games_df.shape[0]

## Adding team pitching stats

In [180]:
train_ds.add_team_pitching_stats(cols=['WHIP'])

,date,Y,M,D,home_team,away_team,home_win,home_pitcher,away_pitcher,home_elo,...,obp_pct_diff,slg_pct_diff,home_rest,away_rest,away_team_season_game_num,home_team_season_game_num,home_W-L-pct_offset1year,away_W-L-pct_offset1year,home_WHIP_offset1year,away_WHIP_offset1year
0,2001-04-01,2001,4.0,1.0,TOR,TEX,1.0,loaizes01,helliri01,1499.562988,...,-2.977845,4.989568,5.0,5.0,0,0,0.512346,0.438272,1.513465,1.640308
1,2001-04-02,2001,4.0,2.0,SEA,OAK,1.0,garcifr03,hudsoti01,1519.463989,...,0.331871,-3.705210,5.0,5.0,0,0,0.561728,0.565217,1.440466,1.498153
2,2001-04-02,2001,4.0,2.0,NYA,KCA,1.0,clemero02,suppaje01,1529.510986,...,1.970596,5.554343,5.0,5.0,0,0,0.540373,0.475309,1.428973,1.582934
3,2001-04-02,2001,4.0,2.0,CIN,ATL,0.0,harnipe01,burkejo03,1527.274048,...,-0.506960,4.555242,5.0,5.0,0,0,0.524691,0.586420,1.445642,1.327686
4,2001-04-02,2001,4.0,2.0,CHN,WAS,0.0,liebejo01,vazquja01,1462.510010,...,2.803560,-4.646432,5.0,5.0,0,0,0.401235,NaN,1.487416,1.512428
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9708,2004-10-03,2004,10.0,3.0,CIN,PIT,0.0,clausbr01,perezol01,1455.857056,...,-16.740494,0.045767,1.0,1.0,80,80,0.425926,0.462963,1.499205,1.405027
9709,2004-10-03,2004,10.0,3.0,OAK,ANA,1.0,hudsoti01,seleaa01,1528.114990,...,-17.270956,-42.810459,1.0,1.0,80,80,0.592593,0.475309,1.273245,1.348613
9710,2004-10-03,2004,10.0,3.0,CHN,ATL,1.0,maddugr01,byrdpa01,1536.317017,...,11.871174,1.509434,1.0,1.0,80,81,0.543210,0.623457,1.319278,1.359797
9711,2004-10-03,2004,10.0,3.0,MIN,CLE,0.0,lohseky01,leecl02,1532.850952,...,1.314286,-8.076923,1.0,1.0,80,80,0.555556,0.419753,1.319463,1.355630


In [181]:
assert train_ds.data.shape[0] == mlb_games_df.shape[0]

## Add pitcher game stats

In [199]:
df = train_ds.data.copy()

In [200]:
df.head()

,date,Y,M,D,home_team,away_team,home_win,home_pitcher,away_pitcher,home_elo,...,avg_diff,obp_diff,slg_diff,avg_pct_diff,obp_pct_diff,slg_pct_diff,home_rest,away_rest,away_team_season_game_num,home_team_season_game_num
0,2001-04-01,2001,4.0,1.0,TOR,TEX,1.0,loaizes01,helliri01,1499.562988,...,-0.008060,-0.010103,0.023271,-2.947374,-2.977845,4.989568,5.0,5.0,0,0
1,2001-04-02,2001,4.0,2.0,SEA,OAK,1.0,garcifr03,hudsoti01,1519.463989,...,-0.000864,0.001190,-0.016229,-0.323318,0.331871,-3.705210,5.0,5.0,0,0
2,2001-04-02,2001,4.0,2.0,NYA,KCA,1.0,clemero02,suppaje01,1529.510986,...,-0.010188,0.006929,0.024787,-3.703559,1.970596,5.554343,5.0,5.0,0,0
3,2001-04-02,2001,4.0,2.0,CIN,ATL,0.0,harnipe01,burkejo03,1527.274048,...,0.003972,-0.001729,0.020216,1.459194,-0.506960,4.555242,5.0,5.0,0,0
4,2001-04-02,2001,4.0,2.0,CHN,WAS,0.0,liebejo01,vazquja01,1462.510010,...,-0.010158,0.009335,-0.018992,-3.996340,2.803560,-4.646432,5.0,5.0,0,0


In [201]:
pitchers_df = pd.read_csv('../data/starting_pitchers_games.csv')
pitchers_df['Date'] = pd.to_datetime(pitchers_df['Date'])
pitchers_df['Year'] = pitchers_df['Date'].dt.year

In [202]:
pitchers_df['season_game'] = pitchers_df.groupby(['name', 'Year'])['Date'].rank('min')
pitchers_df['season_game_offset1'] = pitchers_df['season_game'] + 1
df['home_pitcher_season_game'] = df.groupby(['home_pitcher', 'Y'])['date'].rank('min')

In [203]:
pitchers_df.head()

,Gcar,Gtm,Date,Tm,Opp,Inngs,Dec,DR,IP,H,...,WHIP,Result,Tm_Score,Opp_Score,name,DFS(DK),DFS(FD),Year,season_game,season_game_offset1
0,498,57,2000-06-05,ARI,CHC,GS-5,NaN,5,4.2,5,...,1.904762,L,3,4,morgami01,NaN,NaN,2000,1.0,2.0
1,501,64,2000-06-13,ARI,LAD,GS-5,L(1-1),2,4.2,8,...,2.380952,L,1,6,morgami01,NaN,NaN,2000,2.0,3.0
2,506,79,2000-06-30,ARI,CIN,GS-5,L(3-2),2,5.0,8,...,2.000000,L,4,5,morgami01,NaN,NaN,2000,3.0,4.0
3,512,97,2000-07-21,ARI,CIN,GS-5,NaN,2,5.0,10,...,2.400000,W,5,4,morgami01,NaN,NaN,2000,4.0,5.0
4,541,11,2001-04-14,ARI,COL,GS-4,NaN,2,4.0,8,...,2.000000,L,8,9,morgami01,NaN,NaN,2001,1.0,2.0


In [204]:
df.head()

,date,Y,M,D,home_team,away_team,home_win,home_pitcher,away_pitcher,home_elo,...,obp_diff,slg_diff,avg_pct_diff,obp_pct_diff,slg_pct_diff,home_rest,away_rest,away_team_season_game_num,home_team_season_game_num,home_pitcher_season_game
0,2001-04-01,2001,4.0,1.0,TOR,TEX,1.0,loaizes01,helliri01,1499.562988,...,-0.010103,0.023271,-2.947374,-2.977845,4.989568,5.0,5.0,0,0,1.0
1,2001-04-02,2001,4.0,2.0,SEA,OAK,1.0,garcifr03,hudsoti01,1519.463989,...,0.001190,-0.016229,-0.323318,0.331871,-3.705210,5.0,5.0,0,0,1.0
2,2001-04-02,2001,4.0,2.0,NYA,KCA,1.0,clemero02,suppaje01,1529.510986,...,0.006929,0.024787,-3.703559,1.970596,5.554343,5.0,5.0,0,0,1.0
3,2001-04-02,2001,4.0,2.0,CIN,ATL,0.0,harnipe01,burkejo03,1527.274048,...,-0.001729,0.020216,1.459194,-0.506960,4.555242,5.0,5.0,0,0,1.0
4,2001-04-02,2001,4.0,2.0,CHN,WAS,0.0,liebejo01,vazquja01,1462.510010,...,0.009335,-0.018992,-3.996340,2.803560,-4.646432,5.0,5.0,0,0,1.0


In [205]:
merged_df = df.merge(pitchers_df, left_on=['home_pitcher', 'Y', 'home_pitcher_season_game'],
             right_on=['name', 'Year', 'season_game_offset1'], how='left')

In [206]:
assert merged_df.shape[0] == mlb_games_df.shape[0]

In [207]:
merged_df[merged_df['date'] > '2001-05-01'].head(10)

,date,Y,M,D,home_team,away_team,home_win,home_pitcher,away_pitcher,home_elo,...,WHIP,Result,Tm_Score,Opp_Score,name,DFS(DK),DFS(FD),Year,season_game,season_game_offset1
383,2001-05-02,2001,5.0,2.0,NYN,HOU,0.0,appieke01,limajo01,1516.921021,...,1.142857,W,9.0,4.0,appieke01,NaN,NaN,2001.0,2.0,3.0
384,2001-05-02,2001,5.0,2.0,TEX,DET,0.0,oliveda02,holtch01,1476.520996,...,2.500000,W,7.0,5.0,oliveda02,NaN,NaN,2001.0,2.0,3.0
385,2001-05-02,2001,5.0,2.0,WAS,ARI,1.0,armasto02,anderbr02,1460.550049,...,1.372549,L,2.0,4.0,armasto02,NaN,NaN,2001.0,2.0,3.0
386,2001-05-02,2001,5.0,2.0,SEA,BOS,1.0,seleaa01,nomohi01,1546.921021,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
387,2001-05-02,2001,5.0,2.0,KCA,CLE,0.0,suzukma01,sabatc.01,1484.293945,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
388,2001-05-02,2001,5.0,2.0,BAL,TBA,0.0,mercejo02,rupery01,1481.817017,...,1.147541,L,3.0,4.0,mercejo02,NaN,NaN,2001.0,1.0,2.0
389,2001-05-02,2001,5.0,2.0,PIT,SFN,0.0,ritchto01,ortizru01,1471.198975,...,1.568627,L,2.0,8.0,ritchto01,NaN,NaN,2001.0,2.0,3.0
390,2001-05-02,2001,5.0,2.0,MIN,NYA,1.0,maysjo01,hernaor01,1494.797974,...,1.000000,L,3.0,5.0,maysjo01,NaN,NaN,2001.0,3.0,4.0
391,2001-05-02,2001,5.0,2.0,CHN,SDN,1.0,liebejo01,jarvike01,1477.108032,...,1.285714,W,7.0,6.0,liebejo01,NaN,NaN,2001.0,2.0,3.0
392,2001-05-02,2001,5.0,2.0,MIA,SLN,0.0,pennybr01,morrima01,1487.578979,...,1.166667,L,4.0,5.0,pennybr01,NaN,NaN,2001.0,2.0,3.0


We expect there to be no join when it's the pitchers first start. But if someone has more than one missing value that means they're not in the pitchers data.

In [209]:
merged_df[merged_df['name'].isna()]['home_pitcher'].value_counts()

colonba01    71
clemema01    69
vazquja01    68
sheetbe01    68
clemero02    67
             ..
balfogr01     1
davisla01     1
hillsh01      1
reyesca01     1
kensilo01     1
Name: home_pitcher, Length: 506, dtype: int64

In [211]:
np.sum(merged_df[merged_df['name'].isna()]['home_pitcher'].value_counts() > 1)

338

In [210]:
pitchers_df[pitchers_df['name'] == 'colonba01']

,Gcar,Gtm,Date,Tm,Opp,Inngs,Dec,DR,IP,H,...,WHIP,Result,Tm_Score,Opp_Score,name,DFS(DK),DFS(FD),Year,season_game,season_game_offset1


## Finding missing pitchers

Why are so many pitchers missing from the data?

In [212]:
pitchers_summary_df = pd.read_csv('../data/pitchers_summary.csv')

In [213]:
pitchers_summary_df.head()

,Name,first_season,last_season,games_played,games_started,teams,num_teams,key_mlbam,key_retro,key_bbref,key_fangraphs
0,A.J. Burnett,2000,2015,428,423,"MIA,TOR,NYA,PIT,PHI",5,150359,burna001,burnea.01,512
1,A.J. Cole,2015,2019,79,19,"WAS,CLE",3,595918,colea002,coleaj01,11467
2,A.J. Griffin,2012,2017,88,85,"OAK,TEX",2,456167,grifa002,griffaj01,11132
3,A.J. Murray,2007,2008,16,4,TEX,1,451262,murra001,murraaj01,3422
4,Aaron Blair,2016,2017,16,16,ATL,1,594760,blaia001,blairaa01,14934


In [214]:
pitchers_summary_df[pitchers_summary_df['key_bbref'].str.startswith('colonba')]

,Name,first_season,last_season,games_played,games_started,teams,num_teams,key_mlbam,key_retro,key_bbref,key_fangraphs
127,Bartolo Colon,2000,2018,483,472,"CLE,CHA,ANA,BOS,NYA,athleticsmets,TEX",9,112526,colob001,colonba01,375


## Starting pitchers games

I have three files with pitchers:
- `mlb_games_df.csv` - All games, which also list the starting pitchers
- `pitchers_games.csv` - All games from all pitchers (scraped from BBREF)
- `starting_pitchers_games.csv` - All games from `pitchers_games.csv` where they started (taking from `Inngs` column)

I need to make sure the first and last actually line up.

In [2]:
mlb_df = pd.read_csv('../data/mlb_games_df.csv')
start_df = pd.read_csv('../data/starting_pitchers_games.csv')

In [3]:
mlb_df.head()

,date,Y,M,D,home_team,away_team,home_win,home_pitcher,away_pitcher,home_elo,...,avg_diff,obp_diff,slg_diff,avg_pct_diff,obp_pct_diff,slg_pct_diff,home_rest,away_rest,away_team_season_game_num,home_team_season_game_num
0,2001-04-01,2001,4.0,1.0,TOR,TEX,1.0,loaizes01,helliri01,1499.563,...,-0.008060,-0.010103,0.023271,-2.947374,-2.977845,4.989568,5.0,5.0,0,0
1,2001-04-02,2001,4.0,2.0,SEA,OAK,1.0,garcifr03,hudsoti01,1519.464,...,-0.000864,0.001190,-0.016229,-0.323318,0.331871,-3.705210,5.0,5.0,0,0
2,2001-04-02,2001,4.0,2.0,NYA,KCA,1.0,clemero02,suppaje01,1529.511,...,-0.010188,0.006929,0.024787,-3.703559,1.970596,5.554343,5.0,5.0,0,0
3,2001-04-02,2001,4.0,2.0,CIN,ATL,0.0,harnipe01,burkejo03,1527.274,...,0.003972,-0.001729,0.020216,1.459194,-0.506960,4.555242,5.0,5.0,0,0
4,2001-04-02,2001,4.0,2.0,CHN,WAS,0.0,liebejo01,vazquja01,1462.510,...,-0.010158,0.009335,-0.018992,-3.996340,2.803560,-4.646432,5.0,5.0,0,0


In [4]:
start_df.head()

,Gcar,Gtm,Date,Tm,Opp,Inngs,Dec,DR,IP,H,...,Home_Tm,WHIP,Result,Tm_Score,Opp_Score,name,DFS(DK),DFS(FD),Year,season_game
0,498,57,2000-06-05,ARI,CHC,GS-5,NaN,5,4.2,5,...,CHC,1.904762,L,3,4,morgami01,NaN,NaN,2000,21.0
1,501,64,2000-06-13,ARI,LAD,GS-5,L(1-1),2,4.2,8,...,LAD,2.380952,L,1,6,morgami01,NaN,NaN,2000,24.0
2,506,79,2000-06-30,ARI,CIN,GS-5,L(3-2),2,5.0,8,...,ARI,2.000000,L,4,5,morgami01,NaN,NaN,2000,29.0
3,512,97,2000-07-21,ARI,CIN,GS-5,NaN,2,5.0,10,...,CIN,2.400000,W,5,4,morgami01,NaN,NaN,2000,35.0
4,541,11,2001-04-14,ARI,COL,GS-4,NaN,2,4.0,8,...,COL,2.000000,L,8,9,morgami01,NaN,NaN,2001,4.0


In [5]:
mlb_df.shape[0], start_df.shape[0]

(46159, 96031)

In [6]:
mlb_pitchers = list(set(mlb_df['home_pitcher'].unique()).union(set(mlb_df['away_pitcher'].unique())))

In [7]:
start_pitchers = start_df['name'].unique()

In [8]:
len(set(mlb_pitchers) - set(start_pitchers))

280

In [9]:
len(set(start_pitchers) - set(mlb_pitchers))

364

In [10]:
first_letter = set()
for name in list(set(mlb_pitchers) - set(start_pitchers)):
    first_letter.add(name[0])

In [11]:
first_letter

{'c', 's', 'v'}

In [61]:
list(set(start_pitchers) - set(mlb_pitchers))[:10]

['boehrbr01',
 'robleha01',
 'purgja01',
 'belchti01',
 'lincomi01',
 'ookaa01',
 'merckke01',
 'hwich01',
 'ramsaro01',
 'grosski01']

In [50]:
start_df[start_df['name'] == 'ogelry01']

,Gcar,Gtm,Date,Tm,Opp,Inngs,Dec,DR,IP,H,...,Home_Tm,WHIP,Result,Tm_Score,Opp_Score,name,DFS(DK),DFS(FD),Year,season_game
58749,18,136,2001-09-03,PIT,MIL,GS-3,L(0-4),41,3.0,6,...,PIT,3.000000,L,7,12,ogelry01,NaN,NaN,2001,14.0
58750,19,143,2001-09-09,PIT,CIN,GS-4,L(0-5),5,3.0,4,...,PIT,2.333333,L,3,5,ogelry01,NaN,NaN,2001,15.0
58751,21,72,2003-06-22,PIT,CLE,GS-6,L(0-1),6,5.0,10,...,PIT,2.600000,L,5,8,ogelry01,NaN,NaN,2003,2.0
58752,22,112,2003-08-07,PIT,SFG,GS-3,NaN,45,2.2,4,...,SFG,2.272727,L,5,7,ogelry01,NaN,NaN,2003,3.0
58753,23,149,2003-09-15,PIT,CIN,GS-5,W(1-1),38,5.0,3,...,PIT,1.200000,W,6,3,ogelry01,NaN,NaN,2003,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58922,285,141,2016-09-11,PIT,CIN,GS-4,L(3-5),4,4.0,7,...,PIT,2.500000,L,0,8,ogelry01,NaN,NaN,2016,20.0
58923,286,146,2016-09-16,PIT,CIN,GS-5,NaN,4,4.2,7,...,CIN,2.142857,W,9,7,ogelry01,NaN,NaN,2016,21.0
58924,287,152,2016-09-22,PIT,MIL,GS-4,L(3-6),5,4.0,2,...,MIL,1.000000,L,1,3,ogelry01,NaN,NaN,2016,22.0
58925,288,157,2016-09-27,PIT,CHC,GS-5,L(3-7),4,5.0,4,...,PIT,1.800000,L,4,6,ogelry01,NaN,NaN,2016,23.0


In [14]:
mlb_df[(mlb_df['date'] == '2001-09-03') & (mlb_df['home_team'] == 'PIT')]

,date,Y,M,D,home_team,away_team,home_win,home_pitcher,away_pitcher,home_elo,...,avg_diff,obp_diff,slg_diff,avg_pct_diff,obp_pct_diff,slg_pct_diff,home_rest,away_rest,away_team_season_game_num,home_team_season_game_num
2049,2001-09-03,2001,9.0,3.0,PIT,MIL,1.0,anderji02,queveru01,1437.116,...,0.006814,0.003847,-0.001676,2.507463,1.091645,-0.388812,1.0,1.0,64,63
2055,2001-09-03,2001,9.0,3.0,PIT,MIL,0.0,vogelry01,d'amije01,1439.898,...,0.006814,0.003847,-0.001676,2.507463,1.091645,-0.388812,0.0,0.0,65,64


In [15]:
pitchers_df = pd.read_csv('../data/pitchers_games.csv')
pitchers_df.head()

/Users/psavala/.virtualenvs/baseball/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Gcar,Gtm,Date,Tm,Opp,Inngs,Dec,DR,IP,H,...,Home_Tm,WHIP,Result,Tm_Score,Opp_Score,name,DFS(DK),DFS(FD),Year,season_game
0,478,2,2000-04-05,ARI,PHI,6-GF,S(1),99,4.0,3,...,ARI,0.750000,W,11,3,morgami01,NaN,NaN,2000,1.0
1,479,7,2000-04-10,ARI,SDP,7-GF,S(2),4,3.0,4,...,SDP,1.333333,W,8,4,morgami01,NaN,NaN,2000,2.0
2,480,10,2000-04-13,ARI,SDP,8-GF,S(3),2,2.0,1,...,SDP,0.500000,W,5,4,morgami01,NaN,NaN,2000,3.0
3,481,13,2000-04-17,ARI,COL,6-7,NaN,3,1.2,0,...,ARI,0.000000,L,1,9,morgami01,NaN,NaN,2000,4.0
4,482,15,2000-04-19,ARI,COL,8-8,W(1-0),1,1.0,0,...,ARI,0.000000,W,8,7,morgami01,NaN,NaN,2000,5.0


In [36]:
pitchers_df[pitchers_df['name'] == 'robergr01']

,Gcar,Gtm,Date,Tm,Opp,Inngs,Dec,DR,IP,H,...,Home_Tm,WHIP,Result,Tm_Score,Opp_Score,name,DFS(DK),DFS(FD),Year,season_game
231492,1,99,2000-07-27,NYN,MON,GS-2,NaN,99,1.1,6,...,NYM,8.181818,W,9,8,robergr01,NaN,NaN,2000,1.0
231493,2,151,2000-09-19,NYN,ATL,7-7,NaN,53,1.0,3,...,ATL,4.000000,L,4,12,robergr01,NaN,NaN,2000,2.0
231494,3,160,2000-09-29,NYN,MON,7-7,NaN,9,0.2,1,...,NYM,5.000000,W,11,2,robergr01,NaN,NaN,2000,3.0
231495,4,162,2000-10-01,NYN,MON,3-6,NaN,1,4.0,1,...,NYM,0.250000,W,3,2,robergr01,NaN,NaN,2000,4.0
231496,5,79,2001-06-27,NYN,CHC,3-6,NaN,99,3.1,1,...,CHC,0.322581,W,5,4,robergr01,NaN,NaN,2001,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231563,72,158,2003-09-25,NYN,PIT,6-7,L(0-3),3,1.1,1,...,NYM,1.818182,L,1,3,robergr01,NaN,NaN,2003,18.0
231564,73,2,2004-04-07,NYN,ATL,4-4,NaN,99,0.2,3,...,ATL,25.000000,L,10,18,robergr01,NaN,NaN,2004,1.0
231565,74,7,2004-04-12,NYN,ATL,8-8,NaN,4,1.0,3,...,NYM,4.000000,W,10,6,robergr01,NaN,NaN,2004,2.0
231566,75,8,2004-04-14,NYN,ATL,9-GF,NaN,1,1.0,0,...,NYM,2.000000,L,1,6,robergr01,NaN,NaN,2004,3.0


In [39]:
pitchers_df['Inngs'].unique()

array(['6-GF', '7-GF', '8-GF', '6-7', '8-8', '7-8', '5-6', '10-GF(10)',
       '6-8', '7-7', 'GS-5', '6-6', '4-5', '5-7', '2-6', '9-GF', '2-5',
       '8-9', '8-GF(8)', '4-6', '3-4', '3-6', '10-11', '5-5', 'GS-4',
       '11-GF(11)', '6-GF(8)', '4-7', 'GS-2', 'GS-6', 'GS-3', 'CG(5)',
       'GS-7', '7-9', '5-GF', '13-14', '12-14', '10-GF(13)', '2-8', '5-8',
       '3-5', '4-8', '6-9', 'GS-8', '7-GF(8)', '11-13', '1-6',
       '13-GF(15)', '9-9', '4-4', '14-16', '9-GF(10)', '13-GF(13)',
       '9-10', '10-10', '3-8', '1-3', 'SHO', 'CG', 'GS-9', 'CG(8)',
       'SHO(10)', 'GS-1', 'CG(10)', 'CG(7)', '16-GF(16)', '3-7', '11-11',
       '1-4', '2-4', '10-GF(11)', '12-GF(12)', '3-3', '12-12',
       '14-GF(14)', '12-13', '2-3', '15-GF(15)', '11-GF(12)', '11-12',
       '15-GF(16)', '14-GF(15)', 'SHO(6)', 'CG(6)', '13-13', '1-7',
       '4-GF(8)', '10-13', '14-14', '9-11', '5-GF(8)', '12-GF(14)',
       '12-GF(13)', '10-14', '10-GF(12)', '20-GF(20)', '1-5', '12-17',
       '2-7', '8-GF(10)', 

In [70]:
start_df = pitchers_df[(pitchers_df['Inngs'].str.startswith('GS')) | (pitchers_df['Inngs'].str.startswith('CG'))]

In [71]:
start_df.head(10)

,Gcar,Gtm,Date,Tm,Opp,Inngs,Dec,DR,IP,H,...,Home_Tm,WHIP,Result,Tm_Score,Opp_Score,name,DFS(DK),DFS(FD),Year,season_game
20,498,57,2000-06-05,ARI,CHC,GS-5,NaN,5,4.2,5,...,CHC,1.904762,L,3,4,morgami01,NaN,NaN,2000,21.0
23,501,64,2000-06-13,ARI,LAD,GS-5,L(1-1),2,4.2,8,...,LAD,2.380952,L,1,6,morgami01,NaN,NaN,2000,24.0
28,506,79,2000-06-30,ARI,CIN,GS-5,L(3-2),2,5.0,8,...,ARI,2.000000,L,4,5,morgami01,NaN,NaN,2000,29.0
34,512,97,2000-07-21,ARI,CIN,GS-5,NaN,2,5.0,10,...,CIN,2.400000,W,5,4,morgami01,NaN,NaN,2000,35.0
63,541,11,2001-04-14,ARI,COL,GS-4,NaN,2,4.0,8,...,COL,2.000000,L,8,9,morgami01,NaN,NaN,2001,4.0
155,7,156,2019-09-22,ANA,HOU,GS-2,L(0-1),4,2.0,4,...,HOU,2.500000,L,5,13,rodrijo07,-2.5,0.0,2019,7.0
158,1,80,2017-07-01,MIN,KCR,GS-6,W(1-0),99,5.0,7,...,KCR,1.600000,W,10,5,jorgefe01,NaN,NaN,2017,1.0
159,2,86,2017-07-07,MIN,BAL,GS-3,NaN,5,2.2,7,...,MIN,3.636364,W,9,6,jorgefe01,NaN,NaN,2017,2.0
160,73,1,2000-04-03,MIL,CIN,CG(5),NaN,99,5.0,5,...,CIN,1.200000,T,3,3,woodast01,NaN,NaN,2000,1.0
161,74,6,2000-04-08,MIL,STL,GS-5,L(0-1),4,5.0,10,...,STL,2.800000,L,8,10,woodast01,NaN,NaN,2000,2.0


In [72]:
start_df['Inngs'].unique()

array(['GS-5', 'GS-4', 'GS-2', 'GS-6', 'GS-3', 'CG(5)', 'GS-7', 'GS-8',
       'CG', 'GS-9', 'CG(8)', 'GS-1', 'CG(10)', 'CG(7)', 'CG(6)', 'CG(4)',
       'GS-0', 'GS-10'], dtype=object)

In [73]:
start_df = start_df.reset_index(drop=True)
start_df.to_csv('../data/starting_pitchers_games.csv', index=False)